In [9]:
from Bio import SeqIO
from Bio import SeqFeature
from Bio.Alphabet import IUPAC
import pandas as pd
from Levenshtein import distance
import numpy as np
import csv
from ipywidgets import IntProgress
from IPython.display import display
import multiprocessing as mp
import time
from tqdm import tqdm_notebook as tqdm

# Singlethreaded Code

In [29]:
#get the frequencies for each subfamily in a genome
#ex: get_frequencies("humans")
def get_frequencies(species):
    records = SeqIO.parse("data/" + species + ".fasta", "fasta")
    freq = {}
    for r in records:
        subfamily = r.id.split("_")[2]
        freq[subfamily] = freq.get(subfamily, 0) + 1
    #uncomment these next three lines out if you want the relative frequencies
#     factor=1.0/sum(freq.values())
#     for sub in freq:
#         freq[sub] = freq[sub]*factor
    return freq

#return a list of sequences for a given species and subfamily
#ex: get_subfamily("humans", "AluJo")
def get_subfamily(species, subfamily):
    records = SeqIO.parse("data/" + species + ".fasta", "fasta")
    return [record for record in records if subfamily in record.id]

##return a list of sequences for a given species and subfamily on a chromosome
#ex: get_subfamily_chr("humans", "AluJo", 1)
def get_subfamily_chr(species, subfamily, chromosome):
    records = get_subfamily(species, subfamily)
    return [record for record in records if "chr"+str(chromosome) in record.description]

#retrieve a sequence for a given speicies based on its description
def get_sequence(species, description):
    records = SeqIO.parse("data/" + species + ".fasta", "fasta")
    return [record for record in records if description == record.description]

#parses location information from description string
def get_location(description):
    location = description.split(' ')[1].split(':')
    start, end = location[1].split('-')
    return [location[0].split("=")[1], int(start), int(end)]

def generate_pairings(species1, species2, subfamily):
    species1_records = get_subfamily(species1, subfamily)
    species2_records = get_subfamily(species2, subfamily)
    filename = species1 + "_" + species1 + "_" + subfamily + ".csv"
    file=open(filename,"w+")
    wr = csv.writer(file, quoting=csv.QUOTE_ALL)
    f = IntProgress(min=0, max=len(species1_records))
    display(f)
    for sequence1 in species1_records:
        matches = [distance(str(sequence1.seq), str(sequence2.seq)) for sequence2 in species2_records]
        match = species2_records[np.argmin(matches)]
        location1 = get_location(sequence1.description)
        location2 = get_location(match.description)
        data = np.concatenate([location1, location2, [abs(location1[1] - location2[1])], [str(sequence1.seq)], [str(match.seq)]]) 
        wr.writerow(data)
        f.value += 1
    file.close()

In [25]:
get_frequencies("humans")

{'Alu': 4658,
 'AluJb': 131759,
 'AluJo': 81375,
 'AluJr': 88503,
 'AluJr4': 20966,
 'AluSc': 36338,
 'AluSc5': 7018,
 'AluSc8': 23028,
 'AluSg': 38681,
 'AluSg4': 7603,
 'AluSg7': 9159,
 'AluSp': 53809,
 'AluSq': 19866,
 'AluSq10': 2165,
 'AluSq2': 63875,
 'AluSq4': 1906,
 'AluSx': 123022,
 'AluSx1': 123492,
 'AluSx3': 34198,
 'AluSx4': 11540,
 'AluSz': 107707,
 'AluSz6': 49944,
 'AluY': 110881,
 'AluYa8': 368,
 'AluYd8': 241,
 'AluYe5': 1378,
 'AluYf1': 2025,
 'AluYg6': 899,
 'AluYh9': 165,
 'AluYi6': 470,
 'AluYk11': 1341,
 'AluYk12': 219}

In [ ]:
generate_pairings("humans", "chimps", "AluJo")

# Multithreaded Code

In [ ]:
# set parameters here
species1 = "humans"
species2 = "chimps"
subfamily = "AluJo"

In [ ]:
# then run this cell
print("loading " + species1 + " data...")
species1_records = get_subfamily(species1, subfamily)
print("loading " + species2 + " data...")
species2_records = get_subfamily(species2, subfamily)
generate_pairings()

In [ ]:
def find_match(sequence):
    matches = [distance(str(sequence.seq), str(sequence2.seq)) for sequence2 in species2_records]
    match = species2_records[np.argmin(matches)]
    location1 = get_location(sequence.description)
    location2 = get_location(match.description)
    data = np.concatenate([location1, location2, [abs(location1[1] - location2[1])], [str(sequence.seq)], [str(match.seq)]]) 
    return data
    
def generate_pairings():
    filename = species1 + "_" + species2 + "_" + subfamily + ".csv"
    print("starting matching")
    p = mp.Pool(mp.cpu_count() + 2)
    with open(filename, 'w+') as f:
        wr = csv.writer(f, quoting=csv.QUOTE_ALL)
        with tqdm(total=len(species1_records), unit="match") as pbar:
            for i, result in tqdm(enumerate(p.imap(func=find_match, iterable=species1_records))):
                # (filename, count) tuples from worker
                pbar.update()
                wr.writerow(result)

In [6]:
h = SeqIO.parse("data/" + "humans" + ".fasta", "fasta")

In [7]:
c = SeqIO.parse("data/" + "chimps" + ".fasta", "fasta")

In [8]:
distance(str(next(h).seq),str(next(c).seq))

63